# Neural Network Hidden Layers
As we've seen and discussed, hidden layers are layers in a neural network we don't directly interact with (not input or output layers) and are one of the tools Neural Networks use to overcome linearity (as well as some other hurdles in ML).

In this notebook we'll explore how we can implement a MLP or deep network to solve problems.

----

## Setting Up Our Data
We'll use the same problem as earlier to keep our math simple:

$$ y = X_1+X_2+5 $$

In [ ]:
import numpy as np

np.random.seed(10)

X = np.random.randint(1, 20, size=(1000,2)) # Our inputs

# Our output will be the sum of the array plus five
Y = X.sum(axis=1) + 5

add_intercept = True # Do we want to add an intercept term
if add_intercept:
    # Same thing as with logistic, need a placeholder in front for intercept
    X = np.pad(X, [(0, 0), (1,0)], 'constant', constant_values=1)


print(f'Our feature space:\n{X[:5]}\n')
print(f'Our outputs:\n{Y[:5]}\n')

## Creating our Model
Below is a python class that contains most of the standard features for a Neural Network model. We have functionality to:
 - initializing our model/generate weights
 - feed forward/forward pass
 - backwards propogation
 - training

We'll also work on adding:
 - loss calculation
 - inferencing

And if there is time we may also look at:
 - batch processing
 - modifying our network for multiple types of tasks
 - activation functions

In [126]:
class SimpleNN():
    def __init__(self, input_dim=3, output_dim=1, hidden_layers=[3], lr=0.001):
        self.hidden_layers = hidden_layers
        self.num_hidden_layers = len(hidden_layers)
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weights = []
        self.history = []
        
        self.lr = lr
        self.z = []
        self.y_hat = None

        self.generate_weights()

    def generate_weights(self):
        # For any simple network we need a weight matrix between each layer of the network
        # That means Input -> Hidden Layers -> Output
        num_weights = self.num_hidden_layers + 1
        prev_layer_dim = self.input_dim
        for ix in range(num_weights-1):
            curr_layer_dim = self.hidden_layers[ix]
            curr_weights = np.random.normal(size=(prev_layer_dim, curr_layer_dim))
            self.weights.append(curr_weights)
            prev_layer_dim = curr_layer_dim
        
        curr_weights = np.random.normal(size=(prev_layer_dim, self.output_dim))
        self.weights.append(curr_weights)

    def feed_forward(self, x):
        z = x
        for ix in range(self.num_hidden_layers + 1):
            z = z.dot(self.weights[ix])
            self.z.append(z)
        return z

    def backwards_prop(self, x, y):
        prev_error = y.reshape(-1, 1) - self.z[-1]
        self.history.append(prev_error)
        for ix in range(self.num_hidden_layers + 1)[::-1]:
            w_gradient = -self.z[ix].T.dot(prev_error)
            w_gradient /= y.shape[0]
            prev_error = prev_error.dot(self.weights[ix].T)
            self.weights[ix] -= self.lr * w_gradient

    # TODO: Add logic to print out the MSE at each step of our training
    def train(self, x, y, epochs=10):
        self.z.append(x)
        for _ in range(epochs):
            self.feed_forward(x)
            print(self.mse(x, y))
            self.backwards_prop(x, y)

    # TODO: Implement this function
    def inference(self, x):
        return self.feed_forward(x)
    
    # TODO: Implement this function
    def mse(self, x, y):
        preds = self.feed_forward(x).reshape(-1)
        y_array = np.array(y).reshape(-1)
        return np.round(np.mean((preds - y_array)**2), 2)
    
    def __repr__(self):
        return(f"Model:\
              \n\tInputs - {self.input_dim}\
              \n\tOutputs - {self.output_dim}\
              \n\tLayers - {self.hidden_layers}\
              \n\tWeights - {[x.shape for x in self.weights]}"
              )

In [ ]:
model = SimpleNN(hidden_layers=[3], lr=0.001)
print(model.weights[0])

In [ ]:
model.train(X, Y, epochs=250)

In [ ]:
print(Y[:10])
model.inference(X[:10])

In [ ]:
model.weights